In [1]:
import gpt
from gpt import LanguageModel, block_size, batch_size, eval_iters
import torch
import torch.nn as nn
device = gpt.device
block_size = gpt.block_size
batch_size = gpt.batch_size
eval_iters = gpt.eval_iters
max_iters = gpt.max_iters
eval_interval = gpt.eval_interval

MPS device available


In [2]:
with open('data/eminem/eminem_lyrics/ALL_eminem.txt', 'r', encoding='utf-8') as f:
    eminem = f.read()

print(f"Eminem: {len(eminem)} characters")


Eminem: 925005 characters


In [3]:
with open('data/shake/input.txt', 'r', encoding='utf-8') as f:
    shake = f.read()

with open('data/goethe/full_processed.txt', 'r', encoding='utf-8') as f:
    goethe = f.read()

print(f"Shakespeare: {len(shake)} characters")
print(f"Goethe: {len(goethe)} characters")

goethe = goethe[:len(shake)]  # make them the same length

text = shake + goethe

print(f"Total: {len(text)} characters")

Shakespeare: 1115394 characters
Goethe: 5780194 characters
Total: 2230788 characters


In [4]:
# save the text for later
open('data/shakegoethe.txt', 'w').write(text)

2230788

In [5]:
# Read the texts
with open('data/shake/input.txt', 'r', encoding='utf-8') as f:
    shake_lines = f.readlines()

with open('data/goethe/full_processed.txt', 'r', encoding='utf-8') as f:
    goethe_lines = f.readlines()
print(f"Shakespeare: {len(shake_lines)} lines")
print(f"Goethe: {len(goethe_lines)} lines")

# Ensure both texts have the same number of lines
min_len = min(len(shake_lines), len(goethe_lines))
shake_lines = shake_lines[:min_len]
goethe_lines = goethe_lines[:min_len]
print(f"Total: {min_len} lines")
# Interleave the lines in blocks of 5
blocks = 3
mixed_lines = []
for i in range(0, min_len, blocks):
    mixed_lines.extend(shake_lines[i:i+blocks])  # Add 5 lines from Shakespeare
    mixed_lines.extend(goethe_lines[i:i+blocks])  # Add 5 lines from Goethe

print(f"Total: {len(mixed_lines)} lines")
# Join the lines back into a single text
mixed_text = ''.join(mixed_lines)

# Save the mixed text
with open('data/mixed_shakegoethe.txt', 'w', encoding='utf-8') as f:
    f.write(mixed_text)

# Print the first 500 characters
print(mixed_text[:500])

Shakespeare: 40000 lines
Goethe: 51425 lines
Total: 40000 lines
Total: 80000 lines
First Citizen:
Before we proceed any further, hear me speak.

Wie froh bin ich, daß ich weg bin!
Bester Freund, was ist das Herz  des Menschen!
Dich zu verlassen, den ich so liebe, von dem ich  unzertrennlich war, und froh zu sein!
All:
Speak, speak.

Ich weiß, du verzeihst mir's.
Waren nicht meine übrigen Verbindungen recht ausgesucht vom Schicksal,  um ein Herz wie das meine zu ängstigen?
Die arme Leonore!
First Citizen:
You are all resolved rather to die than to famish?

Und doch  war ich uns


In [6]:
# goethe eminem and shakespear
gemishake = eminem + mixed_text
print(f"Total: {len(gemishake)} characters")
# save the text for later
open('data/mixed_shakegoetheeminem.txt', 'w').write(gemishake)


Total: 7051725 characters


7051725

In [7]:
text = gemishake
textfile = 'data/mixed_shakegoetheeminem.txt'

In [8]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(f"Vocabulary size: {vocab_size}")
# create a mapping of unique characters to integers
char_to_int = {c: i for i, c in enumerate(chars)}
int_to_char = {i: c for i, c in enumerate(chars)}

#encode = lambda s: [char_to_int[c] for c in s]
#decode = lambda l: ''.join([int_to_char[i] for i in l])

Vocabulary size: 127


In [9]:
vocab_size = 30000

In [10]:
# use alternative tokenizer

from tokenizers import Tokenizer
from tokenizers.models import BPE, WordLevel, WordPiece
from tokenizers.trainers import BpeTrainer, WordPieceTrainer
from tokenizers.pre_tokenizers import Whitespace

tokenizer = Tokenizer(WordPiece(unk_token="<unk>"))
tokenizer.pre_tokenizer = Whitespace()
trainer = WordPieceTrainer(vocab_size=vocab_size, special_tokens=["<pad>", "<s>", "</s>", "<unk>"])
tokenizer.train(files=[textfile], trainer=trainer)
tokenizer.save("data/WP_tokenizer.json")

encoded = tokenizer.encode(
    '''To be or not to be,
        that is
        the
        question.
        Als wäre der Himmel über uns
        ein Dach, so sehen wir die Sterne''')
print(encoded.tokens)

decoded = tokenizer.decode(encoded.ids)
print(f'\ndecoded: \n{decoded}')





['To', 'be', 'or', 'not', 'to', 'be', ',', 'that', 'is', 'the', 'question', '.', 'Als', 'wäre', 'der', 'Himmel', 'über', 'uns', 'ein', 'Dach', ',', 'so', 'sehen', 'wir', 'die', 'Sterne']

decoded: 
To be or not to be , that is the question . Als wäre der Himmel über uns ein Dach , so sehen wir die Sterne


In [11]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace

# BPE-Tokenizer
tokenizer = Tokenizer(BPE(unk_token="<unk>"))
tokenizer.pre_tokenizer = Whitespace()
trainer = BpeTrainer(vocab_size=vocab_size)
tokenizer.train(files=[textfile], trainer=trainer)

# Speichern und verwenden
tokenizer.save("data/bpe_tokenizer.json")

encoded = tokenizer.encode(
    '''To be or not to be,
        that is
        the
        question.
        Als wäre der Himmel über uns
        ein Dach, so sehen wir die Sterne''')
print(encoded.tokens)

decoded = tokenizer.decode(encoded.ids)
print(f'\ndecoded: \n{decoded}')




['To', 'be', 'or', 'not', 'to', 'be', ',', 'that', 'is', 'the', 'question', '.', 'Als', 'wäre', 'der', 'Himmel', 'über', 'uns', 'ein', 'Dach', ',', 'so', 'sehen', 'wir', 'die', 'Sterne']

decoded: 
To be or not to be , that is the question . Als wäre der Himmel über uns ein Dach , so sehen wir die Sterne


In [12]:
# use the Worpiece encoder
tokenizer = Tokenizer.from_file("data/bpe_tokenizer.json")

In [13]:
data = torch.tensor(tokenizer.encode(text).ids, dtype=torch.long)
n = int(len(data) * 0.8) # 90% training, 10% validation
train_data, val_data = data[:n], data[n:]

In [14]:
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y



In [15]:
model = LanguageModel(vocab_size).to(device)

In [16]:
print('Parameters:')
print(sum(p.numel() for p in model.parameters()))

Parameters:
33735216


In [17]:
optimizer = torch.optim.Adam(model.parameters(), lr=gpt.learning_rate)

In [18]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [19]:
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


step 0: train loss 10.4599, val loss 10.4545
step 500: train loss 6.7042, val loss 7.0046
step 1000: train loss 6.3937, val loss 6.8075
step 1500: train loss 6.2631, val loss 6.7314
step 2000: train loss 6.1790, val loss 6.7309
step 2500: train loss 5.9902, val loss 6.6270
step 3000: train loss 5.9141, val loss 6.6311
step 3500: train loss 5.7648, val loss 6.6030
step 4000: train loss 5.6505, val loss 6.5852
step 4500: train loss 5.5319, val loss 6.5777
step 4999: train loss 5.4646, val loss 6.6101


In [20]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated = model.generate(context, max_new_tokens=500)  # Generate tokens

# Extract the generated tokens and decode
decoded_text = tokenizer.decode(generated[0].tolist())  # Convert to list and decode
print(decoded_text)
#open('more.txt', 'w').write(decoded_text)

! Da what dances count your patience . ROMEO : Ich hörte zu allen wohl suchen . Es ist rein zur Zeit Ungeheure als eine Hölle und Welle , ob ihn wie von der Freunde macht munter Shady . Von der zweite thou , und häufig for loss i , and enough cke , I count turning worden ing queen you a Span right : daß birds Sie denn sogleich . Du verweilen beteuern Begierde , ließ gesetzt , als der Ebbe innerlich leichter näher zukommen . Grau und wenn Sie kommt , wie dieser Seligkeit ish genug sehen ; es mochte Ritter Teppich . Gestern immer mit dem Bilde , mir don trem sich anwenden Kalkfelsen hätte , vernehmen , Für diesmal und wenn man denn , der Anblick bei keinem kehrte dische an Le Wachen , so subl ver tretend , zu vertilgen es würde davon los sich scharfe letzter ; dergleichen ihre Leidenschaften land Vorwürfe Tränen , wie sie beharrte , nicht haben noch trösten , ins Gedränge nach den höchsten Ehrentitel vorwärts zu haben ; ich nicht und warum hren an meinem Wei , kam gro her zend ; dagegen 

In [21]:
context = "what is the meaning of life? The meaning of life is"
context = torch.tensor(tokenizer.encode(context).ids, dtype=torch.long).unsqueeze(0).to(device)
generated = model.generate(context, max_new_tokens=100)
decoded_text = tokenizer.decode(generated[0].tolist())
print(decoded_text)

what is the meaning of life ? The meaning of life is ! But that lords at is here did , nein unto thy living as I did I get Oxford sh Besuchs he ' s daughter . Durch Der Papier und sah , Und so sieht sich mich irgendwo , Daß gut and an ik prove . Ay , und wider eine hab ' s die Dirne , und dabei wird , als er blitz per , Ob dahin ungen zu unserer , daß sie durch großen Erinnerung daran und verschoben bin enth not bewe ; als people , and low a Kummer am fär present und Jugend an meine Seele


In [22]:
context = "Was ist der Sinn des Lebens? Der Sinn des Lebens ist "
context = torch.tensor(tokenizer.encode(context).ids, dtype=torch.long).unsqueeze(0).to(device)
generated = model.generate(context, max_new_tokens=100)
decoded_text = tokenizer.decode(generated[0].tolist())
print(decoded_text)

Was ist der Sinn des Lebens ? Der Sinn des Lebens ist , die übrigen , unsere , Maultiere du sogleich , So erging K äus Schonung know in response Band belebt . Nun , warum , was das Haus leute , Ach ! Schon fluchte ich mit Vergnügen . Captain . Vor bedeckt und üble schuldig , daß es Ihnen gelte Volks gestraft fäh des D Eduarden Fourth genossen einge Wonne zu suchen . KING LEWIS XI aber nicht meistens war ich bei mir endlich on leuchten alte Prä Werke , getragen , das so hle es mehr verlassen , Dann wurde her als ge ; Die Lügen bt feucht um


In [23]:
context = "I'm beginning to feel like a Rap God, Rap God"
context = torch.tensor(tokenizer.encode(context).ids, dtype=torch.long).unsqueeze(0).to(device)
generated = model.generate(context, max_new_tokens=100)
decoded_text = tokenizer.decode(generated[0].tolist())
print(decoded_text)

I ' m beginning to feel like a Rap God , Rap God ' er - pee the energy ' til Wait with us cit Tape In em feel like ! check fühlte Now and them dying fair demands bum Einsamen a lot ie Jüngl moment of 8 , bout ad ' til that one if you can suck I happy to greet trying window a fling appro ' Til I just be your nut Rei . Re ser me Sie machen shit in einer neuen Söhne , insofern pig that ass Sie bound ppin , auf dem ich von dem Vorbei Pflichten ter Beifall gar liebgewonnen bitten umsehe gemacht und hatte ,
